In [41]:
import sys, pickle, json, os, re
import numpy as np
from collections import defaultdict
from pprint import pprint
from typing import List, Dict, Any
sys.path.append('../src/')
sys.path.append("../../scripts/")
from utils import find_repo_root

import matplotlib.pyplot as plt
from plt_config import use_latex, TU_GREY, TU_RED
use_latex()
project_root_dir = find_repo_root()
print("Project root directory:", project_root_dir)

Project root directory: /home/etschgi1/REPOS/Masterarbeit


In [42]:
ray_folder = "/home/etschgi1/REPOS/Masterarbeit/3_studies/Block_guessing/6-31g_md_testing/tune_logs/MGNN_hyp_small_md_full_mat_loss.py"
trials = []
for folder in os.listdir(ray_folder):
    sub_folder_path = os.path.join(ray_folder, folder)
    if os.path.exists(os.path.join(sub_folder_path, "eval_res_reeval.json")):
        with open(os.path.join(sub_folder_path, "eval_res_reeval.json")) as f:
            eval_res = json.load(f)
        with open(os.path.join(sub_folder_path, "params.json")) as f:
            params = json.load(f)
        with open(os.path.join(sub_folder_path, "eval_res.json")) as f:
            eval_res_old = json.load(f)
        eval_res["cummulative_stats"]["iterations"] = eval_res_old["cummulative_stats"]["iterations"]
        with open(os.path.join(sub_folder_path, "model.history"), "rb") as f:
            hist = pickle.load(f)
            epochs_trained = len(hist["train_loss"])
        params["folder"] = folder
        params["epochs_trained"] = int(epochs_trained)
        trials.append((params, eval_res))


In [43]:
trials.sort(key=lambda x:x[1]["cummulative_stats"]["iterations"]["mean"])
print(f"Found {len(trials)} trials in {ray_folder}")
for trial in trials[:10]: 
    print(trial[0]["folder"])
    # print(trial[1]["cummulative_stats"])
    # print(trial[0]["epochs_trained"])
    # print("---")


Found 10 trials in /home/etschgi1/REPOS/Masterarbeit/3_studies/Block_guessing/6-31g_md_testing/tune_logs/MGNN_hyp_small_md_full_mat_loss.py
hyperopt_train_e6b2b_00029_29_batch_size=8,data_aug_factor=3,edge_threshold_val=2.6495,hidden_dim=512,lr=0.0001,message_net_dropou_2025-07-17_01-07-11
hyperopt_train_e6b2b_00003_3_batch_size=8,data_aug_factor=4,edge_threshold_val=3.2486,hidden_dim=256,lr=0.0004,message_net_dropout_2025-07-16_19-38-52
hyperopt_train_e6b2b_00036_36_batch_size=8,data_aug_factor=2,edge_threshold_val=3.8592,hidden_dim=512,lr=0.0002,message_net_dropou_2025-07-17_02-47-54
hyperopt_train_e6b2b_00022_22_batch_size=16,data_aug_factor=4,edge_threshold_val=3.1101,hidden_dim=128,lr=0.0003,message_net_dropo_2025-07-16_23-57-44
hyperopt_train_e6b2b_00002_2_batch_size=8,data_aug_factor=3,edge_threshold_val=2.2164,hidden_dim=128,lr=0.0005,message_net_dropout_2025-07-16_19-16-11
hyperopt_train_e6b2b_00015_15_batch_size=32,data_aug_factor=3,edge_threshold_val=3.3076,hidden_dim=512,lr

In [44]:
top = 10
table_data = []
for i, (params, metrics) in enumerate(trials[:top]):
    metrics = metrics["cummulative_stats"]
    row = {
        'ID': i,
        'Iterations': f"{float(metrics['iterations']['mean']):.1f}({float(metrics['iterations']['std']):.1f})",
        r"$\Delta E_\text{HF}$": f"{float(metrics['energy_abs']['mean']):.1f}({float(metrics['energy_abs']['std']):.1f})",
        r"$\delta E_\text{HF}$": f"{float(metrics['energy_rel']['mean']):.3f}({float(metrics['energy_rel']['std']):.3f})",
        'DIIS error': f"{float(metrics['diis']['mean']):.3f}({float(metrics['diis']['std']):.3f})",
        r"$\overline{\text{RMSE}}$": f"{float(metrics['rmse']['mean']):.4f}({float(metrics['rmse']['std']):.4f})"
    }
    table_data.append(row)

import pandas as pd
df = pd.DataFrame(table_data)

latex = df.to_latex(index=False)
print(latex)

\begin{tabular}{rlllll}
\toprule
ID & Iterations & $\Delta E_\text{HF}$ & $\delta E_\text{HF}$ & DIIS error & $\overline{\text{RMSE}}$ \\
\midrule
0 & 10.9(0.5) & -1.5(1.7) & -0.001(0.001) & 0.027(0.003) & 0.0075(0.0004) \\
1 & 11.0(0.4) & -1.8(1.3) & -0.001(0.001) & 0.030(0.003) & 0.0077(0.0004) \\
2 & 11.2(0.6) & -3.1(2.2) & -0.002(0.001) & 0.031(0.003) & 0.0080(0.0005) \\
3 & 11.2(0.4) & -1.9(2.1) & -0.001(0.001) & 0.033(0.003) & 0.0084(0.0003) \\
4 & 11.4(0.6) & 3.5(1.2) & 0.002(0.001) & 0.031(0.003) & 0.0081(0.0003) \\
5 & 11.6(1.0) & 0.6(2.3) & 0.000(0.001) & 0.035(0.003) & 0.0087(0.0005) \\
6 & 11.6(0.9) & 0.0(1.8) & 0.000(0.001) & 0.035(0.009) & 0.0088(0.0016) \\
7 & 11.8(0.9) & 1.0(3.3) & 0.001(0.002) & 0.032(0.004) & 0.0085(0.0004) \\
8 & 11.8(0.7) & -4.1(2.0) & -0.002(0.001) & 0.037(0.003) & 0.0086(0.0003) \\
9 & 11.8(0.7) & 1.0(1.8) & 0.001(0.001) & 0.035(0.006) & 0.0083(0.0009) \\
\bottomrule
\end{tabular}



In [45]:
table_data = []
for i, (params, metrics) in enumerate(trials[:top]):
    metrics = metrics["cummulative_stats"]
    row = {
        'Parameter': int(i),
        'Hidden Dimension': int(params["hidden_dim"]),
        'Batch Size': int(params["batch_size"]),
        "Data aug.": f'{float(params["data_aug_factor"]):.2f}',
        "Edge thres.": f'{float(params["edge_threshold_val"]):.2f}',
        "Message p. steps": int(params["message_passing_steps"]),
        "Message Net Dropout": f'{float(params["message_net_dropout"]):.2f}',
        "Message Net Layers": int(params["message_net_layers"]),
        "Learn Rate": f'{float(params["lr"]):.2e}',
        "Weight Decay": f'{float(params["weight_decay"]):.2e}',
    }
    table_data.append(row)

import pandas as pd
df = pd.DataFrame(table_data).T


latex = df.to_latex(index=True)
print(latex)

\begin{tabular}{lllllllllll}
\toprule
 & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 \\
\midrule
Parameter & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 \\
Hidden Dimension & 512 & 256 & 512 & 128 & 128 & 512 & 512 & 512 & 256 & 128 \\
Batch Size & 8 & 8 & 8 & 16 & 8 & 32 & 32 & 16 & 32 & 8 \\
Data aug. & 3.00 & 4.00 & 2.00 & 4.00 & 3.00 & 3.00 & 3.00 & 3.00 & 2.00 & 1.00 \\
Edge thres. & 2.65 & 3.25 & 3.86 & 3.11 & 2.22 & 3.31 & 3.86 & 3.95 & 2.29 & 3.82 \\
Message p. steps & 3 & 4 & 3 & 3 & 4 & 2 & 2 & 2 & 5 & 4 \\
Message Net Dropout & 0.14 & 0.24 & 0.02 & 0.05 & 0.16 & 0.20 & 0.10 & 0.27 & 0.06 & 0.02 \\
Message Net Layers & 3 & 3 & 4 & 4 & 4 & 2 & 3 & 2 & 3 & 3 \\
Learn Rate & 1.08e-04 & 3.99e-04 & 1.95e-04 & 3.27e-04 & 4.82e-04 & 8.66e-04 & 5.65e-04 & 2.80e-04 & 1.21e-03 & 3.32e-03 \\
Weight Decay & 7.13e-04 & 1.24e-04 & 2.35e-04 & 1.50e-05 & 8.57e-04 & 1.35e-04 & 4.36e-05 & 2.06e-04 & 9.34e-04 & 7.39e-04 \\
\bottomrule
\end{tabular}

